In [ ]:
import pandas as pd
import numpy as np

from pyincore import IncoreClient
from pyincore.analyses.socialvulnerability import SocialVulnerability
from pyincore_data.censusutil import CensusUtil
from pyincore import Dataset

# Social vulnerability analysis for Douglas, Kansas census tracts (2019)

In [ ]:
# Parameters
state = "kansas"
county = "douglas"
year = 2019

In [ ]:
# get fips code to use fetch census data
fips = CensusUtil.get_fips_by_state_county(state=state, county=county)
state_code = fips[:2]
county_code = fips[2:]
fips

### construct the Social vulnerability demographic factors

In [ ]:
def demographic_factors(state_number, county_number, year, geo_type="tract:*"):

    _, df_1 = CensusUtil.get_census_data(state=state_code, county=county_code, year=year,
                                                              data_source="acs/acs5",
                                                              columns="GEO_ID,B03002_001E,B03002_003E",
                                                              geo_type=geo_type)
    df_1["factor_white_nonHispanic"] = df_1[["B03002_001E","B03002_003E"]].astype(int).apply(lambda row: row["B03002_003E"]/row["B03002_001E"], axis = 1)

    _, df_2 = CensusUtil.get_census_data(state=state_code, county=county_code, year=year,
                                                  data_source="acs/acs5",
                                                  columns="B03002_001E,B03002_002E",
                                                  geo_type=geo_type)
    df_2["factor_owner_occupied"] = df_2.astype(int).apply(lambda row: row["B03002_002E"]/row["B03002_001E"], axis = 1)
    
    _, df_3 = CensusUtil.get_census_data(state=state_code, 
                                         county=county_code, 
                                         year=year,
                                         data_source="acs/acs5",
                                         columns="B17021_001E,B17021_002E",
                                         geo_type=geo_type)
    df_3["factor_earning_higher_than_national_poverty_rate"] = df_3.astype(int).apply(lambda row: 1-row["B17021_002E"]/row["B17021_001E"], axis = 1)
    
    _, df_4 = CensusUtil.get_census_data(state=state_code, 
                                         county=county_code, 
                                         year=year,
                                         data_source="acs/acs5",
                                         columns="B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E",
                                         geo_type=geo_type)
    df_4["factor_over_25_with_high_school_diploma_or_higher"] = df_4.astype(int).apply(lambda row: (row["B15003_017E"] 
                                                                                                    + row["B15003_018E"] 
                                                                                                    + row["B15003_019E"] 
                                                                                                    + row["B15003_020E"] 
                                                                                                    + row["B15003_021E"] 
                                                                                                    + row["B15003_022E"] 
                                                                                                    + row["B15003_023E"] 
                                                                                                    + row["B15003_024E"] 
                                                                                                    + row["B15003_025E"])/row["B15003_001E"], axis = 1)

    if geo_type == 'tract:*':
        _, df_5 = CensusUtil.get_census_data(state=state_code, 
                                             county=county_code, 
                                             year=year,
                                             data_source="acs/acs5",
                                             columns="B18101_001E,B18101_011E,B18101_014E,B18101_030E,B18101_033E",
                                             geo_type=geo_type)
        df_5["factor_without_disability_age_18_to_65"] = df_5.astype(int).apply(lambda row: (row["B18101_011E"] + row["B18101_014E"] + row["B18101_030E"] + row["B18101_033E"])/row["B18101_001E"], axis = 1)
    
    elif geo_type == 'block%20group:*':
        _, df_5 = CensusUtil.get_census_data(state=state_code, 
                                             county=county_code, 
                                             year=year,
                                             data_source="acs/acs5",
                                             columns="B01003_001E,C21007_006E,C21007_009E,C21007_013E,C21007_016E",
                                             geo_type=geo_type)

        df_5['factor_without_disability_age_18_to_65'] = df_5.astype(int).apply(lambda row: (row['C21007_006E']+
                                                                                 row['C21007_006E']+
                                                                                 row['C21007_009E']+
                                                                                 row['C21007_013E'])
                                                                                /row['C21007_016E'], axis = 1)

    df_t = pd.concat([df_1[["GEO_ID","factor_white_nonHispanic"]],
                      df_2["factor_owner_occupied"],
                      df_3["factor_earning_higher_than_national_poverty_rate"], 
                      df_4["factor_over_25_with_high_school_diploma_or_higher"],
                      df_5["factor_without_disability_age_18_to_65"]], 
                     axis=1, join='inner')
    
    df_t = df_t.set_index("GEO_ID", drop=True)
    
    return df_t

In [ ]:
def national_ave_values (state_code, county_code, year, data_source="acs/acs5", geo_type="tract:*"):
    _, nav1 = CensusUtil.get_census_data(state=state_code, county=county_code, year=year, data_source=data_source,
                             columns="B03002_001E,B03002_003E",geo_type=geo_type)
    nav1 = nav1.astype(int)
    nav1['NAV-1: White, nonHispanic'] = nav1.apply(lambda row: row['B03002_003E']/
                                                  row['B03002_001E'], axis = 1)
    nav1_avg = {"feature": "NAV-1: White, nonHispanic", 
                "average": nav1['NAV-1: White, nonHispanic'].mean()}

    _, nav2 = CensusUtil.get_census_data(state=state_code, county=county_code, year=year, data_source=data_source,
                             columns="B25003_001E,B25003_002E",geo_type=geo_type)
    nav2 = nav2.astype(int)
    nav2['NAV-2: Home Owners'] = nav2.apply(lambda row: row['B25003_002E']/row['B25003_001E'], axis = 1)
    nav2_avg = {"feature": "NAV-2: Home Owners", 
                "average": nav2['NAV-2: Home Owners'].mean()}

    _, nav3 = CensusUtil.get_census_data(state=state_code, county=county_code, year=year, data_source=data_source,
                             columns="B17021_001E,B17021_002E",geo_type=geo_type)
    nav3 = nav3.astype(int)
    nav3['NAV-3: earning higher than national poverty rate'] = nav3.apply(lambda row: 1-(row['B17021_002E']
                                                                                        /row['B17021_001E']), axis = 1)
    nav3_avg = {"feature":'NAV-3: earning higher than national poverty rate', 
                "average": nav3['NAV-3: earning higher than national poverty rate'].mean()}

    _, nav4 = CensusUtil.get_census_data(state=state_code, 
                                         county=county_code, 
                                         year=year,
                                         data_source="acs/acs5",
                                         columns="B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E",
                                         geo_type=geo_type)
    nav4 = nav4.astype(int)
    nav4['NAV-4: over 25 with high school diploma or higher'] = nav4.apply(lambda row: (row['B15003_017E']+
                                                                                       row['B15003_018E']+
                                                                                       row['B15003_019E']+
                                                                                       row['B15003_020E']+
                                                                                       row['B15003_021E']+
                                                                                       row['B15003_022E']+
                                                                                       row['B15003_023E']+
                                                                                       row['B15003_024E']+
                                                                                       row['B15003_025E'])
                                                                                       /row['B15003_001E'], axis = 1)
    nav4_avg = {"feature": 'NAV-4: over 25 with high school diploma or higher', 
                "average": nav4['NAV-4: over 25 with high school diploma or higher'].mean()}

    _, nav5 = CensusUtil.get_census_data(state=state_code, county=county_code, year=year, data_source=data_source,
                             columns="B18101_001E,B18101_011E,B18101_014E,B18101_030E,B18101_033E",
                                         geo_type=geo_type)
    nav5 = nav5.astype(int)
    nav5['NAV-5: without disability age 18 to 65'] = nav5.apply(lambda row: (row['B18101_011E']+
                                                                             row['B18101_014E']+
                                                                             row['B18101_030E']+
                                                                             row['B18101_033E'])/row['B18101_001E'], axis = 1)
    nav5_avg = {"feature": 'NAV-5: without disability age 18 to 65', 
                "average": nav5['NAV-5: without disability age 18 to 65'].mean()}
    
    navs = [nav1_avg, nav2_avg, nav3_avg, nav4_avg, nav5_avg]
    
    return navs

In [ ]:
navs = national_ave_values(state_code, county_code, year)
national_vulnerability_feature_averages = Dataset.from_csv_data(navs, name="national_vulnerability_feature_averages.csv",
                                                                 data_type="incore:socialVulnerabilityFeatureAverages")

In [ ]:
geo_type = "tract:*"
social_vunlnerability_dem_factors_df = demographic_factors(state_code, county_code, year=2019, geo_type=geo_type)
social_vunlnerability_dem_factors = Dataset.from_dataframe(social_vunlnerability_dem_factors_df, 
                                                           name="social_vunlnerability_dem_factors",
                                                           data_type="incore:socialVulnerabilityDemFactors")

In [ ]:
# Configure pyincore access and the analysis with the information above

client = IncoreClient()
social_vulnerability = SocialVulnerability(client)

social_vulnerability.set_parameter("result_name", "social_vulnerabilty")
social_vulnerability.set_input_dataset("national_vulnerability_feature_averages", national_vulnerability_feature_averages)
social_vulnerability.set_input_dataset("social_vulnerability_demographic_factors", social_vunlnerability_dem_factors)

In [ ]:
# Run social vulnerability damage analysis
result = social_vulnerability.run_analysis()

In [ ]:
# Retrieve result dataset
result = social_vulnerability.get_output_dataset("sv_result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()